<a href="https://colab.research.google.com/github/AndrewDavidRatnam/SpamClassification/blob/main/SpamClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SPAM CLASSIFICATION
[Dataset](https://spamassassin.apache.org/old/publiccorpus/)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## Data Extraction

### Reduntant

In [ ]:
# """import tarfile
# def ExtractDataFrom_tarbz2(Filepath):
#   try:
#     with tarfile.open(Filepath, "r:bz2") as tar:
#       tar.extractall(path="") # current directory
#       return "File Extracted Successfully"
#   except Exception as e:

# ExtractDataFrom_tarbz2("20050311_spam_2.tar.bz2")"""

# """import os
# import email
# import csv

# email_data = []

# def get_single_email_data(message):
#       subject = message["Subject"] #print("Subject:", message["Subject"])
#       body = message.get_payload() #print("Body:", body)
#       for part in message.walk():
#         if part.get_content_maintype() == "application" and part.get("Content-Disposition"):
#           filename = part.get_filename()
#       sender = message["From"]
#       recipients_to = message["To"]
#       recipients_cc = message["Cc"]
#       recipients_bcc = message["Bcc"]
#       date = message["Date"]
#       try:
#         return {
#         "Attachment": filename,
#         "Date": date,
#         "Subject": subject,
#         "Sender": sender,
#         "Recipients_To": recipients_to,
#         "Recipients_Cc": recipients_cc,
#         "Recipients_Bcc": recipients_bcc,
#         "Body": body
#     }
#       except:
#         return {
#           "Attachment": "NO ATTACHMENTS",
#           "Date": date,
#           "Subject": subject,
#           "Sender": sender,
#           "Recipients_To": recipients_to,
#           "Recipients_Cc": recipients_cc,
#           "Recipients_Bcc": recipients_bcc,
#           "Body": body
#       }



# for filename in os.listdir("/content/spam_2"):
#   i = 1
#   filepath = os.path.join("/content/spam_2", filename)
#   with open(filepath, "r", encoding="latin1") as file:
#       message = email.message_from_file(file)
#       email_data.append(get_single_email_data(message))
#       #headers = message.items()
#       #print(headers)

#       """

# """csv_file_path = "/content/email_data.csv"
# df = pd.DataFrame(email_data)
# df.to_csv(csv_file_path, index=False)"""


### .

In [1]:
import tarfile
from pathlib import Path
import urllib.request
def fetch_spam_data():   #Retrieves the data from Spam Assasin and returns the paths it was downloaded to
    spam_root = "http://spamassassin.apache.org/old/publiccorpus/"
    ham_url = spam_root + "20030228_easy_ham.tar.bz2"
    spam_url = spam_root + "20030228_spam.tar.bz2"

    spam_path = Path() / "datasets" / "spam"
    spam_path.mkdir(parents=True, exist_ok=True)
    for dir_name, tar_name, url in (("easy_ham", "ham", ham_url),
                                    ("spam", "spam", spam_url)):
        if not (spam_path / dir_name).is_dir():
            path = (spam_path / tar_name).with_suffix(".tar.bz2")
            print("Downloading", path)
            urllib.request.urlretrieve(url, path)
            tar_bz2_file = tarfile.open(path)
            tar_bz2_file.extractall(path=spam_path)
            tar_bz2_file.close()
    return [spam_path / dir_name for dir_name in ("easy_ham", "spam")]

In [2]:
ham_dir, spam_dir = fetch_spam_data()

In [3]:
ham_filenames = [f for f in sorted(ham_dir.iterdir()) if len(f.name) > 20]
spam_filenames = [f for f in sorted(spam_dir.iterdir()) if len(f.name) > 20]
len(ham_filenames), len(spam_filenames)

(2500, 500)

In [4]:
import email
import email.policy

def load_email(filepath):
  with open(filepath, "rb") as f:
    return email.parser.BytesParser(policy=email.policy.default).parse(f)


In [5]:
ham_emails = [load_email(filepath) for filepath in ham_filenames]
spam_emails = [load_email(filepath) for filepath in spam_filenames]

In [6]:
print(ham_emails[1].get_content().strip())

Martin A posted:
Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the
 limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the
 Mount Athos monastic community, was ideal for the patriotic sculpture. 
 
 As well as Alexander's granite features, 240 ft high and 170 ft wide, a
 museum, a restored amphitheatre and car park for admiring crowds are
planned
---------------------
So is this mountain limestone or granite?
If it's limestone, it'll weather pretty fast.

------------------------ Yahoo! Groups Sponsor ---------------------~-->
4 DVDs Free +s&p Join Now
http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM
---------------------------------------------------------------------~->

To unsubscribe from this group, send an email to:
forteana-unsubscribe@egroups.com

 

Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/


In [7]:
print(spam_emails[6].get_content().strip())

Help wanted.  We are a 14 year old fortune 500 company, that is
growing at a tremendous rate.  We are looking for individuals who
want to work from home.

This is an opportunity to make an excellent income.  No experience
is required.  We will train you.

So if you are looking to be employed from home with a career that has
vast opportunities, then go:

http://www.basetel.com/wealthnow

We are looking for energetic and self motivated people.  If that is you
than click on the link and fill out the form, and one of our
employement specialist will contact you.

To be removed from our link simple go to:

http://www.basetel.com/remove.html


4139vOLW7-758DoDY1425FRhM1-764SMFc8513fCsLl40


In [11]:
def get_email_structure(email):
  if isinstance(email, str):
    return email
  payload = email.get_payload()
  if isinstance(payload, list):
    multipart = ", ".join([get_email_structure(sub_email) for sub_email in payload])
    return f"multipart({multipart})"
  else:
    return email.get_content_type()

In [12]:
from collections import Counter

def structures_counter(emails):
  structures = Counter()
  for email in emails:
    structure = get_email_structure(email)
    structures[structure] += 1
  return structures

In [13]:
structures_counter(ham_emails).most_common()

[('text/plain', 2408),
 ('multipart(text/plain, application/pgp-signature)', 66),
 ('multipart(text/plain, text/html)', 8),
 ('multipart(text/plain, text/plain)', 4),
 ('multipart(text/plain)', 3),
 ('multipart(text/plain, application/octet-stream)', 2),
 ('multipart(text/plain, text/enriched)', 1),
 ('multipart(text/plain, application/ms-tnef, text/plain)', 1),
 ('multipart(multipart(text/plain, text/plain, text/plain), application/pgp-signature)',
  1),
 ('multipart(text/plain, video/mng)', 1),
 ('multipart(text/plain, multipart(text/plain))', 1),
 ('multipart(text/plain, application/x-pkcs7-signature)', 1),
 ('multipart(text/plain, multipart(text/plain, text/plain), text/rfc822-headers)',
  1),
 ('multipart(text/plain, multipart(text/plain, text/plain), multipart(multipart(text/plain, application/x-pkcs7-signature)))',
  1),
 ('multipart(text/plain, application/x-java-applet)', 1)]

In [14]:
structures_counter(spam_emails).most_common()

[('text/plain', 218),
 ('text/html', 183),
 ('multipart(text/plain, text/html)', 45),
 ('multipart(text/html)', 20),
 ('multipart(text/plain)', 19),
 ('multipart(multipart(text/html))', 5),
 ('multipart(text/plain, image/jpeg)', 3),
 ('multipart(text/html, application/octet-stream)', 2),
 ('multipart(text/plain, application/octet-stream)', 1),
 ('multipart(text/html, text/plain)', 1),
 ('multipart(multipart(text/html), application/octet-stream, image/jpeg)', 1),
 ('multipart(multipart(text/plain, text/html), image/gif)', 1),
 ('multipart/alternative', 1)]

Clear difference between the structure of emails of spam and easy ham. So structure is an important piece of information.

In [15]:
for header, value in spam_emails[0].items():
  print(header, " : ", value)

Return-Path  :  <12a1mailbot1@web.de>
Delivered-To  :  zzzz@localhost.spamassassin.taint.org
Received  :  from localhost (localhost [127.0.0.1])	by phobos.labs.spamassassin.taint.org (Postfix) with ESMTP id 136B943C32	for <zzzz@localhost>; Thu, 22 Aug 2002 08:17:21 -0400 (EDT)
Received  :  from mail.webnote.net [193.120.211.219]	by localhost with POP3 (fetchmail-5.9.0)	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 13:17:21 +0100 (IST)
Received  :  from dd_it7 ([210.97.77.167])	by webnote.net (8.9.3/8.9.3) with ESMTP id NAA04623	for <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 13:09:41 +0100
From  :  12a1mailbot1@web.de
Received  :  from r-smtp.korea.com - 203.122.2.197 by dd_it7  with Microsoft SMTPSVC(5.5.1775.675.6);	 Sat, 24 Aug 2002 09:42:10 +0900
To  :  dcek1a1@netsgo.com
Subject  :  Life Insurance - Why Pay More?
Date  :  Wed, 21 Aug 2002 20:31:57 -1600
MIME-Version  :  1.0
Message-ID  :  <0103c1042001882DD_IT7@dd_it7>
Content-Type  :  text/html; charset="iso-8859-1"
Cont

Spilting the Data into Test and Train sets

In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

X = np.array(ham_emails + spam_emails, dtype=object)
y = np.array([0]*len(ham_emails) + [1]*len(spam_emails)) #0->ham; 1->spam

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #42 magic number that makes things alright? XD


EMAIL --> DATA PREPROCESS PIPELINE --> TRANSFORMER(EMAIL) --> [TEXT, URLS, PREPROCESS(remove punctutaions replace numbers stemming etc)] -->MATRIX(CSR VOCAB) <--DATA PREPROCESS PIPELINE --> MACHINE_LEARNING --> MODEL

Convert HTML To Plain text, ideally use beautiful soup

In [17]:
#using regex to find and strip html

import re
from html import unescape

def html_to_plain_text(html):
  text = re.sub("<head.*?>.*?</head>", "", html, flags=re.M | re.S | re.I)
  text = re.sub("<style.*?>.*?</style>", "", text, flags=re.M | re.S | re.I) # can use this to see similary styled spam pages
  text = re.sub("<script.*?>.*?</script>", "", text, flags=re.M | re.S | re.I) #can check for scripts and comments also
  text = re.sub("<!--.*?-->", "", text, flags=re.M | re.S)
  text = re.sub("<a\s.*?>", " HYPERLINK ", text, flags = re.M | re.S | re.I)
  text = re.sub("<.*?>", "", text, flags=re.M | re.S)
  text = re.sub(r'(\s*\n)+', " \n ", text, flags=re.M | re.S)
  return unescape(text)

In [18]:
spam_sample = [email for email in X_train[y_train==1] if get_email_structure(email) == "text/html"][7]
print(spam_sample.get_content().strip()[:1000], "...")

<HTML><HEAD><TITLE></TITLE><META http-equiv="Content-Type" content="text/html; charset=windows-1252"><STYLE>A:link {TEX-DECORATION: none}A:active {TEXT-DECORATION: none}A:visited {TEXT-DECORATION: none}A:hover {COLOR: #0033ff; TEXT-DECORATION: underline}</STYLE><META content="MSHTML 6.00.2713.1100" name="GENERATOR"></HEAD>
<BODY text="#000000" vLink="#0033ff" link="#0033ff" bgColor="#CCCC99"><TABLE borderColor="#660000" cellSpacing="0" cellPadding="0" border="0" width="100%"><TR><TD bgColor="#CCCC99" valign="top" colspan="2" height="27">
<font size="6" face="Arial, Helvetica, sans-serif" color="#660000">
<b>OTC</b></font></TD></TR><TR><TD height="2" bgcolor="#6a694f">
<font size="5" face="Times New Roman, Times, serif" color="#FFFFFF">
<b>&nbsp;Newsletter</b></font></TD><TD height="2" bgcolor="#6a694f"><div align="right"><font color="#FFFFFF">
<b>Discover Tomorrow's Winners&nbsp;</b></font></div></TD></TR><TR><TD height="25" colspan="2" bgcolor="#CCCC99"><table width="100%" border="0" 

In [19]:
print(html_to_plain_text(spam_sample.get_content())[:1000], "...")

 
 OTC 
  Newsletter 
 Discover Tomorrow's Winners  
 For Immediate Release 
 Cal-Bay (Stock Symbol: CBYI) 
 Watch for analyst "Strong Buy Recommendations" and several advisory newsletters picking CBYI.  CBYI has filed to be traded on the OTCBB, share prices historically INCREASE when companies get listed on this larger trading exchange. CBYI is trading around 25 cents and should skyrocket to $2.66 - $3.25 a share in the near future. 
 Put CBYI on your watch list, acquire a position TODAY. 
 REASONS TO INVEST IN CBYI 
 A profitable company and is on track to beat ALL earnings estimates! 
 One of the FASTEST growing distributors in environmental & safety equipment instruments. 
 Excellent management team, several EXCLUSIVE contracts.  IMPRESSIVE client list including the U.S. Air Force, Anheuser-Busch, Chevron Refining and Mitsubishi Heavy Industries, GE-Energy & Environmental Research. 
 RAPIDLY GROWING INDUSTRY 
 Industry revenues exceed $900 million, estimates indicate that there cou

In [22]:
def email_to_text(email):
  html = None
  for part in email.walk():
    ctype = part.get_content_type()
    if not ctype in ("text/plain", "text/html"):
      continue
    try:
      content = part.get_content()
    except: #incase of encoding issues
      content = str(part.get_payload())
    if ctype == "text/plain":
      return content
    else:
      html = content
  if html:
      return html_to_plain_text(html)

In [23]:
print(email_to_text(spam_sample)[:100], "...")

 
 OTC 
  Newsletter 
 Discover Tomorrow's Winners  
 For Immediate Release 
 Cal-Bay (Stock Symbol: ...


Using stemming from NLTK. A more modern approach would be lemmatization

In [24]:
# imports and example of simple stemmer, can use nltk lemmatization
import nltk

stemmer = nltk.PorterStemmer()
for word in ("Computations", "Computation", "Computing", "Computed", "Compute", "Compulsive"):
  print(word, " : ", stemmer.stem(word))
for word in ("Working", "Worked", "Works", "Working", "Work", "Works"):
  print(word, " : ", stemmer.stem(word))


Computations  :  comput
Computation  :  comput
Computing  :  comput
Computed  :  comput
Compute  :  comput
Compulsive  :  compuls
Working  :  work
Worked  :  work
Works  :  work
Working  :  work
Work  :  work
Works  :  work


In [25]:
#ONLY ON COLAB, ELSE PIP INSTALL DEPENDCIES, CHECK README.md for all imports and dependcies
%pip install -q -U urlextract

In [27]:
import urlextract #needs internet, but on colab so no problem
#Example
url_extractor = urlextract.URLExtract()
some_text = "CHECK is it will detect some websites like wikipedia.com and https://www.youtube.com/watch?v=YwkyPP111Lc"
print(url_extractor.find_urls(some_text))

['wikipedia.com', 'https://www.youtube.com/watch?v=YwkyPP111Lc']


BaseEstimator: This class serves as the foundation for all estimators (models) in scikit-learn. It provides essential functionalities like:
- Setting and getting parameters of the model using get_params and set_params methods.
- Standardizing the output format for estimators.
- Basic input validation.
<br>
<br>
TransformerMixin: This is a mixin class, meaning it's meant to be inherited along with another class. It's specifically designed for transformers, which are a type of estimator that transforms data.  Key features of TransformerMixin include:
- fit_transform method: This combines the fit (learning parameters from data) and transform (applying the learned transformation) steps into a single method for convenience.
- set_output method: This allows you to specify the desired output container type for the transformed data.
- By importing these classes, you can create custom estimators and transformers that adhere to the scikit-learn API conventions, making them compatible with other scikit-learn tools like pipelines and grid search.

Staring with the custom pipeline components/tasks which will wraped in sklearn.make_pipeline

In [29]:
from sklearn.base import BaseEstimator, TransformerMixin

class Email2WordCounterTransformer(BaseEstimator, TransformerMixin):
  def __init__(self, strip_headers=True, lower_case=True,
               remove_punctuation=True, replace_urls=True,
               replace_numbers=True, stemming=True):
    self.strip_headers = strip_headers
    self.lower_case = lower_case
    self.remove_punctuation = remove_punctuation
    self.replace_urls = replace_urls
    self.replace.numbers = replace_numbers
    self.stemming = stemming

  def fit(self, X, y=None):
    return self #? self.fit?

  def transform(self, X, y=None):
    X_transformed = []
    for email in X:
      text = email_to_text(email) or ""
      if self.lower_case:
        text = text.lower()
      if self.replace_urls and url_extractor is not None: # USEFUL FEATURE
        urls = list(set(url_extractor.find_urls(text)))
        urls.sort(key=lambda url : len(url), reverse=True)
        for url in urls:
          text = text.replace(url, " URL ") # clearly not ideal, can create another column in
          #                                 X and convert urls to vectors and in training similar
          #                                 spam urls will move closer to together with an embedding,
          #                                 can use this as second input in neural network so spam urls move
          #                                  closer in embedding space lol come back and use this feature and try for better results
          if self.replace_numbers:
            text = re.sub(r"\d+(?:\.\d*)?(?:[eE][+-]?\d+)?", "NUMBER", text)
          if self.remove_punctuation:
            text = re.sub(r"\W+", " ", text, flags=re.M)
          word_counts = Counter(text.split())
          if self.stemming and stemmer is not None:
            stemmed_word_counts = Counter()
            for word, count in word_counts.items():
              stemmed_word = stemmer.stem(word)
              stemmed_word_count[stemmed_word] += count
            word_counts = stemmed_word_counts
          X_transformed.append(word_counts)
        return np.array(X_transformed)

### Redundant

In [ ]:
datafile = "/content/email_data.csv"
data = pd.read_csv(datafile)
data.columns

Index(['Attachment', 'Date', 'Subject', 'Sender', 'Recipients_To',
       'Recipients_Cc', 'Recipients_Bcc', 'Body'],
      dtype='object')

### .